LangChain 中的 **MapReduceChain** 是一种用于处理长文本或大量网页的摘要生成方法，其核心原理是通过分块处理和多级摘要来突破大模型上下文长度的限制。以下是其原理、总结及使用思路的详细说明：

---

### **原理**
1. **分块处理（Map 阶段）**  
   将输入的网页内容分割成多个小块（chunk），每个小块独立提交给大模型生成初步摘要。这一过程通过 `Map` 操作实现，每个分块的处理结果作为中间摘要。

2. **合并摘要（Reduce 阶段）**  
   将所有中间摘要组合成一个新的提示词（prompt），再次提交给大模型进行最终摘要生成。这一过程通过 `Reduce` 操作实现，目的是整合各分块的关键信息，形成连贯的总结。

3. **Token 优化**  
   相比直接将全文输入模型（Stuff 方法），MapReduceChain 通过两次摘要减少了单次请求的 Token 开销，但需注意多级处理可能增加整体计算成本。

---

### **总结**
- **优势**：  
  - 适用于长网页或大量网页的摘要，避免超出模型上下文限制。  
  - 通过分块和合并机制，平衡了处理效率与摘要质量。  

- **局限性**：  
  - 多级摘要可能导致信息冗余或关键细节丢失。  
  - Token 开销随分块数量增加而上升，需权衡分块大小。  

---

### **使用思路**
1. **初始化组件**  
   - 加载大模型（如 `ChatOpenAI`）。  
   - 加载网页内容（如使用 `UnstructuredFileLoader` 或 `WebBaseLoader`）。  

2. **选择链类型**  
   使用 `load_summarize_chain` 函数并指定 `chain_type="map_reduce"`，LangChain 会自动构建 MapReduce 链。  

3. **运行链**  
   调用 `chain.run(docs)` 执行摘要生成，输出结果为最终合并的摘要文本。  

4. **代码示例**  
   ```python
   from langchain.chat_models import ChatOpenAI
   from langchain.document_loaders import UnstructuredFileLoader
   from langchain.chains.summarize import load_summarize_chain

   # 初始化模型和网页
   loader = UnstructuredFileLoader("./large_document.pdf")
   docs = loader.load()
   llm = ChatOpenAI(temperature=0)

   # 创建 MapReduce 链
   chain = load_summarize_chain(llm, chain_type="map_reduce")

   # 生成摘要
   summary = chain.run(docs)
   print(summary["output_text"])
   ```

---

### **与其他链类型的对比**
- **Stuff**：直接将全文输入模型，简单但易超限，适合小网页。  
- **Refine**：逐块生成摘要并迭代合并，Token 开销更低，但可能丢失早期信息。  

根据场景需求（如网页长度、Token 限制、摘要质量），选择合适的链类型。

In [32]:
! pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [33]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

In [41]:
# LLM
llm = OllamaLLM(model="deepseek-r1:1.5b", temperature=0)

In [42]:
# Load txt
loader = TextLoader("long_text.txt")
docs = loader.load()

# split text
# Initialize the text splitter with appropriate chunk size and overlap
text_splitter = CharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=10,
    separator="\n",  # Optional: Specify a separator for better splitting
    add_start_index=True  # Optional: Include start indices for tracking
)

split_docs = text_splitter.split_documents(docs)
# print(split_docs)

Created a chunk of size 211, which is longer than the specified 200
Created a chunk of size 218, which is longer than the specified 200
Created a chunk of size 270, which is longer than the specified 200
Created a chunk of size 254, which is longer than the specified 200
Created a chunk of size 242, which is longer than the specified 200
Created a chunk of size 381, which is longer than the specified 200


In [36]:
# map chain
map_template = """对以下内容做简要总结:
"{content}"
总结内容:"""

map_prompt = PromptTemplate(
    input_variables=["content"],
    template=map_template,
)

map_chain = LLMChain(
  llm = llm,
  prompt = map_prompt,
  # verbose = True
)

In [37]:
# reduce chain
reduce_template = """以下是一个摘要集合:
"{summary_content}"
将上述摘要与所有关键细节进行总结。
总结:"""
reduce_prompt = PromptTemplate(
    input_variables=["summary_content"],
    template=reduce_template,
)
reduce_chain = LLMChain(
  llm = llm,
  prompt = reduce_prompt,
  # verbose = True
)

In [38]:
# stuff chain
stuff_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,
    document_variable_name="summary_content",
    # verbose=True,
)

In [39]:
# reduce final chain
reduce_final_chain = ReduceDocumentsChain(
    combine_documents_chain=stuff_chain,
    collapse_documents_chain=stuff_chain,
    token_max=200,
    # verbose=True
)

map_reduce_chain = MapReduceDocumentsChain(
  llm_chain=map_chain,
  document_variable_name="content",
  reduce_documents_chain=reduce_final_chain,
  verbose=True
)

In [43]:
summary = map_reduce_chain.run(split_docs)
print(summary)



> Entering new MapReduceDocumentsChain chain...


KeyboardInterrupt: 